In [1]:
import json

from utils.themes.programme import Programme

In [2]:
from utils.paths import SITE

EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

In [3]:
(
    Programme.events
    .aggregate('Month', len, field='Events')
    .convert('Month', lambda f: f.isoformat())  
).tocsv(EVENTS / 'total_by_month.csv')

In [4]:
(
    Programme.events
    .aggregate(['Project Name', 'Month'], len)
    .recast(key='Project Name', variablefield='Month', missing=0)
).tocsv(EVENTS / 'monthly_by_project.csv')

In [5]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(Programme.events.aggregate('Project Name', len).sort('value', reverse=True).records()),
        f,
        indent=2,
    )

In [6]:
from datetime import date
Programme.excluded_events.selectne('Item Type', 'Event (any public-facing activity)').nrows()

2160

In [7]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': Programme.events.nrows(),
            'excluded': {
                'total': Programme.excluded_events.nrows(),
                'non_event': Programme.excluded_events.selectne('Item Type', 'Event (any public-facing activity)').nrows(),
                'future': Programme.excluded_events.selecteq('Item Type', 'Event (any public-facing activity)').selectge('Date', date.today()).nrows(),
                'data_quality': Programme.excluded_events.selecteq('Item Type', 'Event (any public-facing activity)').selectlt('Date', date.today()).nrows()
            },
            'date': {
                'earliest': min(Programme.events.values('Date')).isoformat(),
                'latest': max(Programme.events.values('Date')).isoformat(),
            }
        },
        f,
        indent=2,
    )